In [1]:
import pandas as pd
from datetime import datetime, timedelta

# Function to generate date ranges for each month
def generate_month_ranges(year):
    start_date = datetime(year, 4, 1)  # Start from April
    end_date = datetime(year + 1, 3, 31)  # End in March of the next year
    # Specify public holidays
    public_holidays = {
        "New Year's Day": datetime(year + 1, 1, 1),
        "Day after New Year's Day": datetime(year + 1, 1, 2),
        "Auckland Anniversary": datetime(year + 1, 1, 29),
        "Waitangi Day": datetime(year + 1, 2, 6),
        "Good Friday": datetime(year + 1, 3, 29),
        "Easter Monday": datetime(year, 4, 1),
        "Anzac Day": datetime(year, 4, 25),
        "King's Birthday": datetime(year, 6, 3),
        "Matariki": datetime(year, 6, 28),
        "Labour Day": datetime(year, 10, 28),
        "Christmas Day": datetime(year, 12, 25),
        "Boxing Day": datetime(year, 12, 26)
    }
    month_ranges = []
    current_date = start_date
    while current_date <= end_date:
        # Calculate the end of the current month
        month_end = datetime(current_date.year, current_date.month, get_days_in_month(current_date.year, current_date.month))
        # Determine the start and end dates for this month
        month_start = current_date if current_date.day == 1 else current_date.replace(day=1)
        month_end = month_end
        # Add the month range to the list
        month_ranges.append((month_start, month_end))
        # Move to the next month
        current_date = month_end + timedelta(days=1)
    return month_ranges, public_holidays
 
# Function to get the number of days within a given month
def get_days_in_month(year, month):
    if month == 12:
        next_month = 1
        next_year = year + 1
    else:
        next_month = month + 1
        next_year = year
    last_day_of_month = datetime(next_year, next_month, 1) - timedelta(days=1)
    return last_day_of_month.day
 
# Create a DataFrame
year = 2024
month_ranges, public_holidays = generate_month_ranges(year)
data = []
 
for i, (month_start, month_end) in enumerate(month_ranges):
    current_date = month_start
    while current_date <= month_end:
        test_week_start = current_date - timedelta(days=current_date.weekday())
        test_week_end = test_week_start + timedelta(days=6)
 
        # Calculate the start and end of the week
        #1 This week does not contain a beginning of a month and an end of the last month at the same time
        if test_week_start.month == current_date.month == test_week_end.month:
            week_start = test_week_start  # Find Monday of the current week
            week_end = test_week_end - timedelta(days=1)  # Find Saturday of the current week
 
        #2 Current day is <= end of month
        if test_week_start.month == current_date.month != test_week_end.month:
            week_start = test_week_start  # Find Monday of the current week
            week_end = month_end  # Find Saturday of the current week
 
        #3 Current day is  >= start of month
        if test_week_start.month != current_date.month == test_week_end.month:
            if current_date == test_week_end == month_start: # If it's 1st of a month while it's Sunday
                week_start = current_date + timedelta(days=1)
            else:
                week_start = month_start  # Find Monday of the current week
                week_end = test_week_end - timedelta(days=1)  # Find Saturday of the current week

        # Calculate the week number                        
        month_str = f"{current_date.strftime('%B')}-{str(current_date.year)[-2:]}"
        last_Sunday = month_start - timedelta(days=month_start.weekday() + 1) 
        if month_start.weekday() == 0:
            week_number = (test_week_end - last_Sunday).days // 7
        if month_start.weekday() == 6:
            week_number = (test_week_end - last_Sunday).days // 7 - 1
        if month_start.weekday() not in [0, 6]:
            week_number = (test_week_end - last_Sunday).days // 7
 
        # Extract the day numbers for Monday to Saturday
        week_start_day = week_start.day
        week_end_day = week_end.day
 
        # Determine if any public holidays fall within this week
        public_holiday_names = [holiday_name for holiday_name, holiday_date in public_holidays.items() if
                                week_start <= holiday_date <= week_end]
        
        # Count the public holidays in the week
        ph_days = len(public_holiday_names)
        
        
        from datetime import datetime, timedelta

        # Calculate the date range for the week (excluding public holidays)
        date_range = ""
        current_day = week_start
        while current_day <= week_end:
            if current_date.replace(day=current_day.day) not in public_holidays.values():
                if date_range and current_day != test_week_end:  # Check if there are already dates in the range and if it's not the end of the week
                    date_range += ", "
                date_range += str(current_day.day)  # Append the day to the date range
                start_of_range = current_day  # Record the start of the range
                while current_day < week_end and current_date.replace(day=current_day.day + 1) not in public_holidays.values():  # Loop until next day is holiday or end of week
                    current_day += timedelta(days=1)  # Move to the next day
                if start_of_range != current_day:  # Check if the range spans multiple days
                    date_range += "-" + str(current_day.day)  # Append the end day of the range
            current_day += timedelta(days=1)  # Move to the next day
 
        data.append([month_str, f"Week {week_number}", date_range,
                     week_end_day - week_start_day - ph_days + 1, ", ".join(public_holiday_names), ph_days,
                     None, None, None, None, None])

        # Move to the next week
        if current_date.weekday() == 5:  
            current_date += timedelta(days=2)  # Move to the next day (Monday) to start the next week
        else:
            current_date = test_week_end + timedelta(days=1)  # Move to the next day after the week ends

# Create DataFrame
columns = ["Month", "Week", "Date Range", "Days", "Public Holidays", "PH-Days", "Weekly Target", "Weekly Actual",
           "Lead Time", "4 Week Rolling Target", "4 Week Rolling Actual"]
df = pd.DataFrame(data, columns=columns)

# Delete the row if date_range is null and week is week 0
df = df.dropna(subset=['Date Range'])  # Drop rows with null date_range
df = df[df['Week'] != 'Week 0']  # Drop rows with week labeled as week 0

# Save to Excel
excel_filename = "2025_financial_year_schedule.xlsx"
df.to_excel(excel_filename, index=False)
print(f"Excel file '{excel_filename}' created successfully.")

Excel file '2025_financial_year_schedule.xlsx' created successfully.
